In [1]:
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import time

# On importe la métrique "silhouette" qui nous servira à déterminer le nombre de clusters optimal.
from sklearn.metrics import silhouette_score 

%config InlineBackend.figure_format = 'png'

t = time.time()

In [2]:
df = pd.read_csv('data/df_train_prepro.csv')

In [3]:
len(df)

1610494

In [4]:
# On enlève des variables ltf grâce aux corrélations visibles dans la matrice. 
df_clusters = df[['contextid','dayssincelastvisitdouble','campaignctrlast24h','nbdisplay_1hour','nbdayssincelastclick','display_size','ltf_nbpartnerdisplayssincelastclick','ltf_nbglobaldisplay_4w','ltf_nbpartnerclick_90d','ltf_nbpartnerdisplay_90d','ltf_nbpartnersales_90d','nbdisplayglobalapprox_1d_sum_xdevice']]

In [5]:
# On va chercher à obtenir des groupes d'observations de caractéristiques proches afin de voir si l'on peut distinguer les clics des non-clics. 
sc = StandardScaler()
Z = sc.fit_transform(df_clusters)
Z

array([[ 0.14873565, -0.40809959, -0.8906595 , ..., -0.07290974,
        -0.01397608, -0.32103262],
       [-1.92104218, -0.44867976, -0.360678  , ..., -0.07290974,
        -0.01397608,  0.8070464 ],
       [ 0.14873565, -0.33748309,  1.82717896, ..., -0.07290974,
        -0.01397608, -0.5009203 ],
       ...,
       [ 0.97664678, -0.36785013, -0.92834265, ..., -0.07290974,
        -0.01397608, -0.51352702],
       [ 1.39060235, -0.4168148 ,  2.20278178, ..., -0.07290974,
        -0.01397608, -0.34134216],
       [ 0.14873565, -0.1664472 , -0.50158415, ..., -0.07290974,
        -0.01397608, -0.45594838]])

In [6]:
# Vérification que les moyennes sont nulles et les écarts-types unitaires. 
print(np.mean(Z, axis=0))
print(np.std(Z, axis=0, ddof=0))

[-4.69520271e-17  2.48472492e-16 -5.90496082e-17 -2.12479762e-17
 -9.95337090e-18  1.23711223e-17  1.66154232e-17 -1.56712648e-17
  9.90263342e-18  4.87962244e-18 -2.46683444e-18 -3.14228272e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
# On fixe une graine aléatoire pour avoir des résultats reproductibles
GRAINE = 146355806

In [8]:
%%time
# On applique la méthode des k-means sur les variables centrées et réduites, en supposant pour l'instant par défaut qu'il existe 2 clusters.
kmeans = KMeans(n_clusters=2, random_state=GRAINE) 
# random_state est la graine aléatoire specifiée pour rendre les résultats reproductibles.
kmeans.fit(Z)

# On trie les index en fonction des différents groupes de députés (ie des clusters).
idk = np.argsort(kmeans.labels_)

# Affichage des observations et de leurs groupes.
print(pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]}))

         Observation  Cluster
0                  0        0
1            1072675        0
2            1072674        0
3            1072673        0
4            1072672        0
...              ...      ...
1610489      1146842        1
1610490       282184        1
1610491      1250993        1
1610492       271392        1
1610493       831799        1

[1610494 rows x 2 columns]
Wall time: 7.49 s


In [9]:
nb_clusters = kmeans.labels_.tolist()
print(nb_clusters.count(0)) # Comptage du nombre de députés dans le cluster 1.
print(nb_clusters.count(1)) # Comptage du nombre de députés dans le cluster 2.

1604453
6041


Le découpage en deux clusters ne nous permettra pas de discriminer les clics des non-clics ; le deuxième cluster est en effet trop petit. 
Essayons avec 3 clusters. 

In [10]:
%%time
# Cellule dédiée au choix de la taille de l'échantillon du cluster sur lequel on va calculer le score de silhouette.
# On utilise également la graine GRAINE pour l'échantillonnage.
silhouette_score(Z, kmeans.labels_, n_jobs=-1, sample_size=25000, random_state=GRAINE)

Wall time: 13.5 s


0.8680866579433779

In [11]:
%%time
# On va faire varier le nombre de clusters dans un intervalle réaliste, disons de 2 à 10.
range_clusters = np.arange(2, 11)
res = []
for k in range_clusters:
    kmeans = KMeans(n_clusters=k, random_state=GRAINE)
    kmeans.fit(Z)
    res.append(silhouette_score(Z, kmeans.labels_, n_jobs=-1, sample_size=25000, random_state=GRAINE))
    print(f"Nb clusters : {k} ; score silhouette : {res[-1]:.4f}.")
# print(res)

Nb clusters : 2 ; score silhouette : 0.8681.
Nb clusters : 3 ; score silhouette : 0.2797.
Nb clusters : 4 ; score silhouette : 0.2880.
Nb clusters : 5 ; score silhouette : 0.1795.
Nb clusters : 6 ; score silhouette : 0.2042.
Nb clusters : 7 ; score silhouette : 0.2311.
Nb clusters : 8 ; score silhouette : 0.2498.
Nb clusters : 9 ; score silhouette : 0.2529.
Nb clusters : 10 ; score silhouette : 0.2623.
Wall time: 6min 33s


In [12]:
plt.figure(figsize=(8,4))
plt.plot(range_clusters, res)
plt.title("Coefficient de silhouette moyen en fonction du nombre de clusters", size=16)
plt.xlabel("Nombre de clusters", size=10)
plt.ylabel("Score de silhouette moyen", size=10)
plt.show()

In [13]:
kmeans = KMeans(n_clusters=4, random_state=GRAINE) 
kmeans.fit(Z)
idk = np.argsort(kmeans.labels_)
print(pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]}))

         Observation  Cluster
0                  0        0
1             998281        0
2             998280        0
3             998279        0
4             998278        0
...              ...      ...
1610489      1199080        3
1610490      1199078        3
1610491       925303        3
1610492       310208        3
1610493       858440        3

[1610494 rows x 2 columns]


In [14]:
nb_clusters1 = kmeans.labels_.tolist()
print(nb_clusters1.count(0)) 
print(nb_clusters1.count(1))
print(nb_clusters1.count(2))
print(nb_clusters1.count(3))

1214530
6005
171724
218235


Etudions les caractéristiques des observations dans chacun des clusters. 

In [15]:
A = pd.DataFrame({"Observation" : df_clusters.index[idk], "Cluster" : kmeans.labels_[idk]})
L1 = [] # Liste qui contiendra les index des députés du cluster 1.
L2 = [] # Liste qui contiendra les index des députés du cluster 2.
L3 = [] # Liste qui contiendra les index des députés du cluster 3.
L4 = [] # Liste qui contiendra les index des députés du cluster 4.
for i in range(len(A)) :
    if A['Cluster'][i] == 0 :
        L1.append(A['Observation'][i])
    if A['Cluster'][i] == 1 :
        L2.append(A['Observation'][i])
    if A['Cluster'][i] == 2 : 
        L3.append(A['Observation'][i])
    if A['Cluster'][i] == 3 : 
        L4.append(A['Observation'][i])

In [16]:
df_cluster_1 = df.loc[L1] # Sous-table avec les députés du cluster 1.
df_cluster_2 = df.loc[L2] # Sous-table avec les députés du cluster 2.
df_cluster_3 = df.loc[L3] # Sous-table avec les députés du cluster 3.
df_cluster_4 = df.loc[L4] # Sous-table avec les députés du cluster 4.

In [17]:
len(L1), len(L2), len(L3), len(L4)

(1214530, 6005, 171724, 218235)

On retrouve bien le nombre d'observations présentes dans chaque cluster. 

In [18]:
# Répartition des clics et des non clics dans le cluster 1.
display(pd.DataFrame({'Nb dans cluster 1' : df_cluster_1['is_display_clicked'].value_counts(), 'Prop dans cluster 1' : df_cluster_1['is_display_clicked'].value_counts(normalize=True)})) 
# Répartition des clics et des non clics dans le cluster 2.
display(pd.DataFrame({'Nb dans cluster 2' : df_cluster_2['is_display_clicked'].value_counts(), 'Prop dans cluster 2' : df_cluster_2['is_display_clicked'].value_counts(normalize=True)})) 
# Répartition des clics et des non clics dans le cluster 3.
display(pd.DataFrame({'Nb dans cluster 3' : df_cluster_3['is_display_clicked'].value_counts(), 'Prop dans cluster 3' : df_cluster_3['is_display_clicked'].value_counts(normalize=True)}))
# Répartition des clics et des non clics dans le cluster 4.
display(pd.DataFrame({'Nb dans cluster 4' : df_cluster_4['is_display_clicked'].value_counts(), 'Prop dans cluster 4' : df_cluster_4['is_display_clicked'].value_counts(normalize=True)}))

,Nb dans cluster 1,Prop dans cluster 1
0,1137695,0.936737
1,76835,0.063263


,Nb dans cluster 2,Prop dans cluster 2
0,5561,0.926062
1,444,0.073938


,Nb dans cluster 3,Prop dans cluster 3
0,164902,0.960273
1,6822,0.039727


,Nb dans cluster 4,Prop dans cluster 4
0,210584,0.964941
1,7651,0.035059


In [19]:
df_cluster_1.describe()

,Unnamed: 0,Unnamed: 0.1,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.209507e+06,1.209507e+06,1.209507e+06,1.110091e+06,1.110091e+06,1.110091e+06,...,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06,1.214530e+06
mean,9.286841e+05,9.293023e+05,5.477336e+00,1.112730e+00,3.659951e+01,7.285497e-01,9.446224e+01,9.839391e+07,-7.566101e+07,-1.107862e+08,...,1.124536e+01,1.094701e+01,3.081458e+01,2.579208e-01,2.862949e+02,2.252042e+02,1.602857e+09,6.326316e-02,1.192573e+00,8.114627e+04
std,5.348763e+05,5.352814e+05,2.477548e+00,3.162626e-01,3.341509e+01,8.078666e-01,9.783166e+02,1.259963e+09,1.156586e+09,1.173206e+09,...,1.428090e+01,2.099309e+01,3.532637e+01,1.845674e-01,2.497447e+02,2.281093e+02,1.745169e+05,2.434358e-01,3.145508e+00,9.764375e+04
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,-1.000000e+00,-2.147137e+09,-2.147454e+09,-2.147479e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,3.411759e-02,0.000000e+00,0.000000e+00,1.602547e+09,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.664125e+05,4.667155e+05,4.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.200000e+01,-9.654318e+08,-1.003142e+09,-1.022446e+09,...,1.000000e+00,0.000000e+00,6.270266e+00,1.131779e-01,2.000000e+00,2.000000e+00,1.602700e+09,0.000000e+00,8.500000e-02,4.000000e+00
50%,9.280425e+05,9.285535e+05,6.000000e+00,1.000000e+00,2.900000e+01,1.000000e+00,1.600000e+01,2.603107e+08,-6.162982e+07,-2.862254e+08,...,5.864144e+00,3.000000e+00,1.800000e+01,1.859788e-01,3.000000e+02,2.500000e+02,1.602855e+09,0.000000e+00,3.333878e-01,7.500000e+04
75%,1.391986e+06,1.392882e+06,7.000000e+00,1.000000e+00,7.300000e+01,1.000000e+00,1.270000e+02,1.384186e+09,8.233637e+08,7.813881e+08,...,1.544028e+01,1.130546e+01,4.212188e+01,5.000993e-01,3.200000e+02,2.800000e+02,1.603021e+09,0.000000e+00,1.013000e+00,1.024000e+05
max,1.855313e+06,1.856872e+06,9.000000e+00,2.000000e+00,1.340000e+02,2.000000e+00,3.177000e+04,2.147366e+09,2.147451e+09,2.147451e+09,...,1.890000e+02,3.036156e+02,2.505810e+02,7.905422e-01,1.374000e+03,1.286000e+03,1.603155e+09,1.000000e+00,1.000000e+02,1.101600e+06


In [20]:
df_cluster_2.describe()

,Unnamed: 0,Unnamed: 0.1,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,6.005000e+03,6.005000e+03,6005.000000,6005.000000,5988.000000,5988.000000,5988.000000,5.356000e+03,5.356000e+03,5.356000e+03,...,6005.000000,6005.000000,6005.000000,6005.000000,6005.000000,6005.000000,6.005000e+03,6005.000000,6005.000000,6.005000e+03
mean,9.210501e+05,9.216633e+05,7.374188,1.137552,34.011189,0.739813,69.330828,1.356797e+08,-9.248536e+07,-1.410151e+08,...,25.198707,31.044927,82.745036,0.307009,327.961699,242.026978,1.602856e+09,0.073938,1.567122,8.853621e+04
std,5.381714e+05,5.385786e+05,1.842490,0.344458,32.846903,0.794863,563.200537,1.259821e+09,1.138648e+09,1.155639e+09,...,21.056553,65.505627,99.869488,0.189287,266.523156,231.326105,1.767911e+05,0.261692,3.778073,9.751542e+04
min,9.400000e+01,9.400000e+01,1.000000,1.000000,3.000000,0.000000,1.000000,-2.140960e+09,-2.146561e+09,-2.146412e+09,...,0.000000,0.000000,0.000000,0.034118,0.000000,0.000000,1.602550e+09,0.000000,0.008475,0.000000e+00
25%,4.576510e+05,4.579450e+05,6.000000,1.000000,4.000000,0.000000,12.000000,-9.001059e+08,-1.006771e+09,-1.089605e+09,...,6.979398,2.000000,28.000000,0.139050,160.000000,50.000000,1.602698e+09,0.000000,0.118646,1.600000e+04
50%,9.178820e+05,9.183840e+05,8.000000,1.000000,16.000000,1.000000,16.000000,3.640997e+08,-6.162982e+07,-2.701107e+08,...,21.543889,10.179676,54.371852,0.245336,300.000000,250.000000,1.602850e+09,0.000000,0.470166,7.500000e+04
75%,1.389601e+06,1.390497e+06,9.000000,1.000000,73.000000,1.000000,127.000000,1.384186e+09,7.278802e+08,6.550648e+08,...,41.000000,32.000000,98.489537,0.512843,336.000000,300.000000,1.603023e+09,0.000000,1.419046,1.536000e+05
max,1.855146e+06,1.856705e+06,9.000000,2.000000,134.000000,2.000000,30724.000000,2.141561e+09,2.146829e+09,2.144924e+09,...,193.195278,1341.353333,1416.062778,0.790542,1280.000000,1280.000000,1.603149e+09,1.000000,84.459663,1.024000e+06


In [21]:
df_cluster_3.describe()

,Unnamed: 0,Unnamed: 0.1,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,1.717240e+05,1.717240e+05,171724.000000,171724.000000,171006.000000,171006.000000,171006.000000,1.545350e+05,1.545350e+05,1.545350e+05,...,171724.000000,171724.000000,171724.000000,171724.000000,171724.000000,171724.000000,1.717240e+05,171724.000000,171724.000000,1.717240e+05
mean,9.240505e+05,9.246655e+05,6.217273,1.089947,36.273692,0.717647,95.766312,9.122278e+07,-6.187909e+07,-1.034572e+08,...,9.113251,12.982262,32.625727,0.224353,290.448342,219.232099,1.602858e+09,0.039727,0.739145,7.997536e+04
std,5.351861e+05,5.355911e+05,1.647041,0.286106,33.283918,0.810803,1015.665626,1.265114e+09,1.160608e+09,1.174688e+09,...,12.553802,28.416230,44.880125,0.175795,255.211043,225.136794,1.755761e+05,0.195317,1.898668,9.725414e+04
min,1.000000e+01,1.000000e+01,1.000000,1.000000,3.000000,0.000000,-1.000000,-2.147479e+09,-2.147278e+09,-2.147319e+09,...,0.000000,0.000000,0.000000,0.034118,0.000000,0.000000,1.602547e+09,0.000000,0.000000,0.000000e+00
25%,4.592812e+05,4.595772e+05,6.000000,1.000000,4.000000,0.000000,12.000000,-1.015546e+09,-9.653825e+08,-1.009380e+09,...,1.000000,0.000000,6.000000,0.087429,0.000000,0.000000,1.602700e+09,0.000000,0.051818,0.000000e+00
50%,9.215770e+05,9.220810e+05,6.000000,1.000000,29.000000,0.000000,16.000000,2.292188e+08,-6.162982e+07,-2.820157e+08,...,4.000000,3.000000,16.559676,0.144372,300.000000,250.000000,1.602858e+09,0.000000,0.218000,7.500000e+04
75%,1.386480e+06,1.387374e+06,7.000000,1.000000,73.000000,1.000000,127.000000,1.384186e+09,8.401876e+08,7.830828e+08,...,12.000000,12.000000,40.576736,0.299222,320.000000,280.000000,1.603023e+09,0.000000,0.661073,9.600000e+04
max,1.855310e+06,1.856869e+06,9.000000,2.000000,134.000000,2.000000,31770.000000,2.146981e+09,2.147417e+09,2.147359e+09,...,174.481250,716.947917,734.793750,0.790542,1280.000000,1280.000000,1.603154e+09,1.000000,100.000000,1.024000e+06


In [22]:
df_cluster_4.describe()

,Unnamed: 0,Unnamed: 0.1,contextid,target_env,rtbtypeid,rtbadvisibility,rtb_detectedlanguage,urlhash2,urlhash3,urlhash4,...,nbdisplaypartnerapprox_1d_sum_xdevice,nbdisplayaffiliateapprox_1d_sum_xdevice,nbdisplayglobalapprox_1d_sum_xdevice,valueperclick,display_width,display_height,display_timestamp,is_display_clicked,zonecostineuro,display_size
count,2.182350e+05,2.182350e+05,218235.000000,218235.000000,217082.00000,217082.000000,217082.000000,1.895640e+05,1.895640e+05,1.895640e+05,...,218235.000000,218235.000000,218235.000000,218235.000000,218235.000000,218235.000000,2.182350e+05,218235.000000,218235.000000,2.182350e+05
mean,9.315689e+05,9.321890e+05,6.048443,1.072756,37.08740,0.732064,94.816503,1.113809e+08,-1.778726e+08,-1.883800e+08,...,35.948418,59.626699,128.330583,0.313647,363.708369,251.435228,1.602853e+09,0.035059,0.940507,9.208258e+04
std,5.370751e+05,5.374817e+05,2.453060,0.259737,33.75389,0.791542,1046.916814,1.215893e+09,1.124596e+09,1.153969e+09,...,20.215833,89.078501,125.753298,0.189133,277.856972,234.957654,1.775439e+05,0.183928,2.544857,9.393635e+04
min,3.000000e+00,3.000000e+00,1.000000,1.000000,3.00000,0.000000,-1.000000,-2.145916e+09,-2.147307e+09,-2.147255e+09,...,0.000000,0.000000,13.000000,0.034118,0.000000,0.000000,1.602547e+09,0.000000,0.000000,0.000000e+00
25%,4.663500e+05,4.666530e+05,6.000000,1.000000,4.00000,0.000000,12.000000,-8.462803e+08,-1.089605e+09,-1.089605e+09,...,22.000000,11.000000,47.987940,0.136647,300.000000,50.000000,1.602694e+09,0.000000,0.050000,1.600000e+04
50%,9.363980e+05,9.369130e+05,6.000000,1.000000,29.00000,1.000000,12.000000,3.640997e+08,-2.180814e+08,-5.237546e+08,...,35.000000,28.414352,83.436574,0.269752,300.000000,250.000000,1.602849e+09,0.000000,0.258000,7.500000e+04
75%,1.396296e+06,1.397196e+06,8.000000,1.000000,73.00000,1.000000,127.000000,1.335917e+09,6.404254e+08,6.397927e+08,...,49.023519,67.000000,169.923750,0.512843,360.000000,480.000000,1.603021e+09,0.000000,0.830579,1.800000e+05
max,1.855305e+06,1.856864e+06,9.000000,2.000000,134.00000,2.000000,31770.000000,2.146395e+09,2.147353e+09,2.147353e+09,...,460.277870,2269.557407,3953.888889,0.790542,1280.000000,1286.000000,1.603152e+09,1.000000,100.000000,1.028800e+06


In [23]:
print(f"Temps d'exécution total : {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

Temps d'exécution total : 00:08:10
